# VAR modeling 
To assess the relationship between Wallstreetbest sentiment and stock's data, we will use VAR models. 

The VAR model is a statistical model for forecasting time series data. The difference between VAR and other time series models like ARIMA, ARMA or GARCH, is that VAR allows for multivariate analysis, and can also be used with exogenous variables(pretty much timeseries modeling for endogenous and common regression for exogenous).

In [3]:
library(tidyverse);
library(recipes)
library(caret)
library(xts)
library(MTS)
library(tseries)
library(forecast)
library(vars)
library(lmtest)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Caricamento pacchetto: 'recipes'


Il seguente oggetto è mascherato da 'package:stringr':

    fixed


Il seguente oggetto è mascherato da 'package:stats':

    step


Caricamento del pacchetto richiesto: lattice


Caricamento pacchetto: 'caret'


Il seguente oggetto è mascherato da 'package:purrr':

    lift


Caricamento del pacchetto richiesto: zoo


Caricamento pacchetto: 'zoo'


I seguenti oggetti sono mascherati da 'package:base':

    as.Date, as.Date.numeric



Caricamento pacchetto: 'xts'


I seguenti oggetti sono mascherati da 'package:dplyr':

    first, last


Registered S3 method overwritten b

In [2]:
r50 <- read.csv(file = "../data/r50_2023.csv",
                header = TRUE,
                sep = ",",
                dec = ".")

# Center and scale
recipe <-
    recipe(~., data = r50) %>%
    step_center(all_numeric()) %>%
    step_scale(all_numeric())

recipe

recipe <- prep(recipe, training = r50)

r50_clean <- bake(recipe, r50)

       X          timestamp             score          comms_num      
 Min.   :    1   Length:17222       Min.   :     0   Min.   :    0.0  
 1st Qu.: 4306   Class :character   1st Qu.:     4   1st Qu.:    5.0  
 Median : 8612   Mode  :character   Median :    58   Median :   25.0  
 Mean   : 8612                      Mean   :  1321   Mean   :  294.1  
 3rd Qu.:12917                      3rd Qu.:   286   3rd Qu.:   83.0  
 Max.   :17222                      Max.   :225870   Max.   :93268.0  
                                    NA's   :17       NA's   :17       
   sentiment             r               logr             Dvol       
 Min.   :-1.0000   Min.   :-0.053   Min.   :-0.067   Min.   :-0.223  
 1st Qu.:-0.1040   1st Qu.:-0.036   1st Qu.:-0.058   1st Qu.:-0.061  
 Median : 0.4680   Median :-0.003   Median :-0.003   Median :-0.020  
 Mean   : 0.2992   Mean   : 0.000   Mean   :-0.011   Mean   :   Inf  
 3rd Qu.: 0.8730   3rd Qu.: 0.034   3rd Qu.: 0.022   3rd Qu.: 0.061  
 Max.   : 1.

ERROR: Error in recipe(~., data = r50) %>% step_center(all_numeric()) %>% step_scale(all_numeric()): non trovo la funzione "%>%"
